In [41]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, Markdown
import datetime

def choose_time_window(csv_file, date_col='created_at'):
    # Load CSV and convert the selected date column to datetime with UTC
    df = pd.read_csv(csv_file)
    df['__date_filter__'] = pd.to_datetime(df[date_col], utc=True, errors='coerce')
    df = df.dropna(subset=['__date_filter__'])  # Drop invalid date rows

    # Get the full range of data recording
    min_time = df['__date_filter__'].min()
    max_time = df['__date_filter__'].max()

    # Start and end times
    print(f"The dataset starts recording at {min_time.strftime('%Y-%m-%d %H:%M:%S %Z')}")
    print(f"The dataset ends recording at {max_time.strftime('%Y-%m-%d %H:%M:%S %Z')}")

    # Time dropdown options
    hour_options = [f"{h:02d}" for h in range(24)]
    minute_options = [f"{m:02d}" for m in range(60)]
    second_options = [f"{s:02d}" for s in range(60)]

    # Date pickers
    start_date = widgets.DatePicker(description='Start Date', value=min_time.date())
    end_date = widgets.DatePicker(description='End Date', value=max_time.date())

    # Time pickers for start
    start_hour = widgets.Dropdown(description='Hour', options=hour_options, value='00')
    start_minute = widgets.Dropdown(description='Minute', options=minute_options, value='00')
    start_second = widgets.Dropdown(description='Second', options=second_options, value='00')

    # Time pickers for end
    end_hour = widgets.Dropdown(description='Hour', options=hour_options, value='23')
    end_minute = widgets.Dropdown(description='Minute', options=minute_options, value='59')
    end_second = widgets.Dropdown(description='Second', options=second_options, value='59')

    button = widgets.Button(description='Filter')
    output = widgets.Output()

    # Filtering logic on button click
    def on_click(b):
        with output:
            output.clear_output()

            # Combine selected date and time into strings
            start_dt_str = f"{start_date.value} {start_hour.value}:{start_minute.value}:{start_second.value}"
            end_dt_str = f"{end_date.value} {end_hour.value}:{end_minute.value}:{end_second.value}"

            # Convert to timezone-aware datetime objects
            start_ts = pd.to_datetime(start_dt_str).tz_localize('UTC')
            end_ts = pd.to_datetime(end_dt_str).tz_localize('UTC')

            # Filter the dataframe
            data1 = (df['__date_filter__'] >= start_ts) & (df['__date_filter__'] <= end_ts)
            filtered_df = df.loc[data1].copy()

            # Format date column as dd/mm/yy 
            filtered_df[date_col] = filtered_df['__date_filter__'].dt.strftime('%d/%m/%y')
            filtered_df.drop(columns=['__date_filter__'], inplace=True)

            display(filtered_df)

    button.on_click(on_click)

    # Display UI
    ui = widgets.VBox([
        widgets.HBox([start_date, start_hour, start_minute, start_second]),
        widgets.HBox([end_date, end_hour, end_minute, end_second]),
        button,
        output
    ])
    display(ui)


In [42]:
choose_time_window('2881821.csv', date_col='created_at')


The dataset starts recording at 2025-03-18 06:54:26 UTC
The dataset ends recording at 2025-03-19 07:02:42 UTC
